# chronos-t5-large

In [ ]:
import matplotlib.pyplot as plt  # ТОП прогноз + праздничные дни
import numpy as np
import pandas as pd
import torch
from chronos import ChronosPipeline

# Загружаем модель напрямую из Hugging Face
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-large",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)
print(f"PyTorch использует устройство: {torch.cuda.current_device()} (GPU)" if torch.cuda.is_available() else "PyTorch использует устройство: CPU")

# Загружаем данные
data = pd.read_csv('C:/Users/bondarenkovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].astype(str).str.replace(',', '.').astype(float)
data = data.drop_duplicates(subset=['store', 'ds']).dropna()
data = data.sort_values(by=['store', 'ds']).reset_index(drop=True)

# Заполняем пропущенные даты
date_range = pd.date_range(start=data['ds'].min(), end=data['ds'].max(), freq='D')
all_stores = data['store'].unique()
full_index = pd.MultiIndex.from_product([all_stores, date_range], names=['store', 'ds'])
data = data.set_index(['store', 'ds']).reindex(full_index).fillna(0).reset_index()

# Параметры прогнозирования
prediction_length = 40  # Прогноз на 40 дней
last_n_days = 14  # Период для проверки активности магазина (последние 14 дней)

# Определяем активные магазины (те, у которых есть продажи за последние 14 дней)
last_date = data['ds'].max()
active_stores = []
for store in all_stores:
    store_data = data[(data['store'] == store) & (data['ds'] > last_date - pd.Timedelta(days=last_n_days))]
    if store_data['y'].sum() > 0:  # Если сумма продаж за последние 14 дней больше 0
        active_stores.append(store)

print(f"Активные магазины (с продажами за последние {last_n_days} дней): {len(active_stores)} из {len(all_stores)}")

# Список для хранения прогнозов
forecasts = []

# Прогноз только для активных магазинов с выводом подсказок
for store in active_stores:
    print(f"Рассчитывается прогноз для магазина: {store}")
    # Выбираем данные для текущего магазина
    store_data = data[data['store'] == store]['y'].values
    context = torch.tensor(store_data, dtype=torch.float32)  # Контекст — исторические данные
    
    # Генерируем прогноз
    forecast = pipeline.predict(context, prediction_length)  # shape: [1, num_samples, prediction_length]
    forecasts.append(forecast[0].numpy())  # Сохраняем прогноз как numpy массив
    print(f"Прогноз для магазина {store} успешно рассчитан")

# Преобразуем прогнозы в DataFrame
forecast_df = pd.DataFrame({
    'store': active_stores,
    'forecast': [np.median(f, axis=0) for f in forecasts]  # Берем медиану по сэмплам
})

# Разворачиваем прогноз по дням
forecast_days = pd.date_range(start=data['ds'].max() + pd.Timedelta(days=1), periods=prediction_length, freq='D')
forecast_expanded = forecast_df.explode('forecast').reset_index(drop=True)
forecast_expanded['ds'] = forecast_days.tolist() * len(active_stores)

# Добавляем предпраздничные дни вручную
pre_holiday_dates = pd.to_datetime([
    '2025-02-22', '2025-03-07', '2025-04-19', '2025-05-08', '2025-06-11', '2025-04-20', '2025-05-02', 
    '2025-05-03', '2025-12-28', '2025-12-29', '2025-12-30', '2025-12-31'
])
forecast_expanded['is_pre_holiday'] = forecast_expanded['ds'].isin(pre_holiday_dates).astype(int)

# Корректируем прогноз только для предпраздничных дней
forecast_expanded['forecast'] = forecast_expanded['forecast'] * (1 + 0.15 * forecast_expanded['is_pre_holiday'])

# Сохраняем результат
forecast_expanded.to_excel('Chronos_T5_Forecast_with_pre_holidays_large.xlsx', index=False)
print("Прогноз успешно сохранен!")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from chronos import ChronosPipeline
import time

# Функция для выполнения прогноза и замера времени для одного магазина
def run_forecast(device, pipeline, store, data, prediction_length):
    print(f"\nЗапуск прогноза для магазина {store} на устройстве: {device}")
    start_time = time.time()  # Начало замера времени
    
    store_data = data[data['store'] == store]['y'].values
    context = torch.tensor(store_data, dtype=torch.float32).to(device)
    forecast = pipeline.predict(context, prediction_length)
    forecast_result = forecast[0].cpu().numpy()  # Переносим результат на CPU
    
    end_time = time.time()  # Конец замера времени
    elapsed_time = end_time - start_time
    print(f"Прогноз для магазина {store} на {device} выполнен за {elapsed_time:.2f} секунд")
    return forecast_result, elapsed_time

# Загружаем данные
data = pd.read_csv('C:/Users/bondarenkovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].astype(str).str.replace(',', '.').astype(float)
data = data.drop_duplicates(subset=['store', 'ds']).dropna()
data = data.sort_values(by=['store', 'ds']).reset_index(drop=True)

# Заполняем пропущенные даты
date_range = pd.date_range(start=data['ds'].min(), end=data['ds'].max(), freq='D')
all_stores = data['store'].unique()
full_index = pd.MultiIndex.from_product([all_stores, date_range], names=['store', 'ds'])
data = data.set_index(['store', 'ds']).reindex(full_index).fillna(0).reset_index()

# Параметры прогнозирования
prediction_length = 40
last_n_days = 14

# Определяем активные магазины
last_date = data['ds'].max()
active_stores = [store for store in all_stores if data[(data['store'] == store) & (data['ds'] > last_date - pd.Timedelta(days=last_n_days))]['y'].sum() > 0]
print(f"Активные магазины (с продажами за последние {last_n_days} дней): {len(active_stores)} из {len(all_stores)}")

# Выбираем первый активный магазин для теста
test_store = active_stores[0]
print(f"Тестируемый магазин: {test_store}")

# Устройства для тестирования
devices = ["cpu"]
if torch.cuda.is_available():
    devices.append("cuda")
else:
    print("GPU недоступен, тестирование будет выполнено только на CPU")

# Тестирование на CPU и GPU
results = {}
for device_name in devices:
    device = torch.device(device_name)
    print(f"\nНастройка модели для устройства: {device_name}")
    pipeline = ChronosPipeline.from_pretrained(
        "amazon/chronos-t5-large",
        device_map=device_name,
        torch_dtype=torch.bfloat16,
    )
    forecast, elapsed_time = run_forecast(device_name, pipeline, test_store, data, prediction_length)
    results[device_name] = (forecast, elapsed_time)

# Используем результат с последнего устройства (обычно GPU, если доступен) для дальнейшей обработки
forecast = results[devices[-1]][0]

# Преобразуем прогноз в DataFrame (для одного магазина)
forecast_df = pd.DataFrame({
    'store': [test_store],
    'forecast': [np.median(forecast, axis=0)]
})

# Разворачиваем прогноз по дням
forecast_days = pd.date_range(start=data['ds'].max() + pd.Timedelta(days=1), periods=prediction_length, freq='D')
forecast_expanded = forecast_df.explode('forecast').reset_index(drop=True)
forecast_expanded['ds'] = forecast_days.tolist()

# Добавляем предпраздничные дни
pre_holiday_dates = pd.to_datetime([
    '2025-02-22', '2025-03-07', '2025-04-19', '2025-05-08', '2025-06-11', '2025-04-20', '2025-05-02', 
    '2025-05-03', '2025-12-28', '2025-12-29', '2025-12-30', '2025-12-31'
])
forecast_expanded['is_pre_holiday'] = forecast_expanded['ds'].isin(pre_holiday_dates).astype(int)

# Корректируем прогноз
forecast_expanded['forecast'] = forecast_expanded['forecast'] * (1 + 0.15 * forecast_expanded['is_pre_holiday'])

# Сохраняем результат
forecast_expanded.to_excel('Chronos_T5_Forecast_one_store.xlsx', index=False)
print("\nПрогноз для одного магазина успешно сохранен!")

# Вывод сравнения времени
print("\nСравнение времени выполнения для магазина", test_store)
for device_name, (_, elapsed_time) in results.items():
    print(f"{device_name}: {elapsed_time:.2f} секунд")

Активные магазины (с продажами за последние 14 дней): 174 из 185
Тестируемый магазин: 1

Настройка модели для устройства: cpu

Запуск прогноза для магазина 1 на устройстве: cpu


# chronos-bolt-base

In [ ]:
import pandas as pd
import torch
from chronos import BaseChronosPipeline
import numpy as np

# Инициализируем пайплайн
pipeline = BaseChronosPipeline.from_pretrained(
    "amazon/chronos-bolt-base",
    device_map="cuda" if torch.cuda.is_available() else "cpu",
    torch_dtype=torch.bfloat16,
)

print(f"PyTorch использует устройство: {torch.cuda.current_device()} (GPU)" if torch.cuda.is_available() else "PyTorch использует устройство: CPU")

# Загружаем данные
data = pd.read_csv('C:/Users/bondarenkovv/Desktop/Python/NeuralProphet/Magazin/Sales.csv', sep=';', low_memory=False)
data['Дата'] = pd.to_datetime(data['Дата'], dayfirst=True)
data = data.rename(columns={'Дата': 'ds', 'Номер Магазина': 'store', '(Сутки).(Сумма продаж в фактических ценах реализации(валюта))': 'y'})
data['y'] = data['y'].astype(str).str.replace(',', '.').astype(float)
data = data.drop_duplicates(subset=['store', 'ds']).dropna()
data = data.sort_values(by=['store', 'ds']).reset_index(drop=True)

# Заполняем пропущенные даты
date_range = pd.date_range(start=data['ds'].min(), end=data['ds'].max(), freq='D')
all_stores = data['store'].unique()
full_index = pd.MultiIndex.from_product([all_stores, date_range], names=['store', 'ds'])
data = data.set_index(['store', 'ds']).reindex(full_index).fillna(0).reset_index()

# Параметры прогнозирования
prediction_length = 40  # Прогноз на 40 дней
last_n_days = 14  # Период для проверки активности магазина (последние 14 дней)

# Определяем активные магазины (те, у которых есть продажи за последние 14 дней)
last_date = data['ds'].max()
active_stores = []
for store in all_stores:
    store_data = data[(data['store'] == store) & (data['ds'] > last_date - pd.Timedelta(days=last_n_days))]
    if store_data['y'].sum() > 0:  # Если сумма продаж за последние 14 дней больше 0
        active_stores.append(store)

print(f"Активные магазины (с продажами за последние {last_n_days} дней): {len(active_stores)} из {len(all_stores)}")

# Список для хранения прогнозов
forecasts = []

# Прогноз только для активных магазинов с подсказками
for store in active_stores:
    print(f"Рассчитывается прогноз для магазина: {store}")
    store_data = data[data['store'] == store]['y'].values
    context = torch.tensor(store_data, dtype=torch.float32)
    forecast = pipeline.predict(context, prediction_length)
    forecasts.append(forecast[0].numpy())
    print(f"Прогноз для магазина {store} успешно рассчитан")

# Преобразуем прогнозы в DataFrame
forecast_df = pd.DataFrame({
    'store': active_stores,
    'forecast': [np.median(f, axis=0) for f in forecasts]
})

# Разворачиваем прогноз по дням
forecast_days = pd.date_range(start=data['ds'].max() + pd.Timedelta(days=1), periods=prediction_length, freq='D')
forecast_expanded = forecast_df.explode('forecast').reset_index(drop=True)
forecast_expanded['ds'] = forecast_days.tolist() * len(active_stores)

# Убираем отрицательные прогнозы (устанавливаем минимум 0)
forecast_expanded['forecast'] = forecast_expanded['forecast'].clip(lower=0)

# Добавляем предпраздничные дни
pre_holiday_dates = pd.to_datetime([
    '2025-02-22', '2025-03-07', '2025-04-19', '2025-05-08', '2025-06-11', '2025-04-20', '2025-05-02', 
    '2025-05-03', '2025-12-28', '2025-12-29', '2025-12-30', '2025-12-31'
])
forecast_expanded['is_pre_holiday'] = forecast_expanded['ds'].isin(pre_holiday_dates).astype(int)

# Корректируем прогноз для предпраздничных дней
forecast_expanded['forecast'] = forecast_expanded['forecast'] * (1 + 0.15 * forecast_expanded['is_pre_holiday'])

# Сохраняем результат
forecast_expanded.to_excel('Chronos_T5_Forecast_with_pre_holidays.xlsx', index=False)
print("Прогноз успешно сохранен!")

# График

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Настройки для графиков
plt.style.use('seaborn')  # Стиль Seaborn для красивого отображения
plt.rcParams['figure.figsize'] = (14, 6)  # Увеличим ширину для лучшей видимости

# Фильтруем фактические данные с начала 2025 года и считаем сумму по всем магазинам
start_date = pd.to_datetime('2025-01-01')
fact_2025 = data[data['ds'] >= start_date].copy()
fact_total = fact_2025.groupby('ds')['y'].sum().reset_index()

# Считаем суммарный прогноз по всем активным магазинам
forecast_total = forecast_expanded.groupby('ds')['forecast'].sum().reset_index()
forecast_total['is_pre_holiday'] = forecast_total['ds'].isin(pre_holiday_dates).astype(int)

# Определяем пользовательские метки с шагом 7 дней, начиная с 30 декабря 2024
start_xticks = pd.to_datetime('2024-12-30')
end_date = forecast_total['ds'].max()  # Последняя дата прогноза
xticks = pd.date_range(start=start_xticks, end=end_date, freq='7D')

# Построение графика
plt.figure()
# Факт
plt.plot(fact_total['ds'], fact_total['y'], label='Факт (с 2025, сумма по всем магазинам)', color='blue')
# Прогноз
plt.plot(forecast_total['ds'], forecast_total['forecast'], label='Прогноз (сумма по активным магазинам)', color='green', linestyle='--')
# Отмечаем предпраздничные дни на прогнозе
pre_holiday_forecast = forecast_total[forecast_total['is_pre_holiday'] == 1]
plt.scatter(pre_holiday_forecast['ds'], pre_holiday_forecast['forecast'], 
            color='red', label='Предпраздничные дни', zorder=5, s=50)

plt.title('Суммарные продажи: Факт с 2025 года и прогноз')
plt.xlabel('Дата')
plt.ylabel('Сумма продаж')
plt.legend()
plt.grid(True)

# Устанавливаем метки с шагом 7 дней
plt.xticks(xticks, rotation=45, ha='right')  # Метки каждые 7 дней с 30 декабря 2024
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))  # Формат даты
plt.tight_layout()
plt.savefig('Total_fact_2025_forecast_7days_from_2024-12-30.png')  # Сохранение графика
plt.show()

print("График с суммарными фактическими данными с 2025 года и прогнозом с шагом 7 дней построен и сохранен!")